# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [1]:

# Your code here
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# IMDb URL for Oppenheimer reviews
IMDB_URL = "https://www.imdb.com/title/tt15398776/reviews"

# Headers to mimic a real browser request and avoid bot detection
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def scrape_oppenheimer_reviews(num_reviews=1000):
    """
    Function to scrape IMDb reviews for Oppenheimer (2023).
    This script extracts:
    - Review Title
    - Review Text
    """

    review_data = []  # List to store extracted reviews
    page_number = 1   # Initialize page number for tracking

    while len(review_data) < num_reviews:
        try:
            # Sending request to IMDb page
            response = requests.get(IMDB_URL, headers=HEADERS)
            if response.status_code != 200:
                print(f"Failed to retrieve page {page_number}. Status Code: {response.status_code}")
                break  # Stop execution if request fails

            # Parsing HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # **Find all review containers**
            review_elements = soup.find_all('div', class_='sc-8c7aa573-5 gBEznl')  # Updated class

            # If no reviews found, exit loop
            if not review_elements:
                print("No more reviews found. IMDb structure might have changed.")
                break

            # Extract review title and review text
            for review in review_elements:
                try:
                    # Extracting Review Title
                    title_tag = review.find("h3", class_="ipc-title__text")
                    review_title = title_tag.text.strip() if title_tag else "No Title"

                    # Extracting Review Text
                    text_tag = review.find("div", class_="ipc-html-content-inner-div")
                    review_text = text_tag.text.strip() if text_tag else "No Text Available"

                    # Checking if full text is available in an alternate location
                    if review_text == "No Text Available":
                        text_alt_tag = review.find_next_sibling("div", {"data-testid": "review-overflow"})
                        review_text = text_alt_tag.text.strip() if text_alt_tag else "No Text"

                    # Appending extracted review details to list
                    review_data.append({
                        "Review Title": review_title,
                        "Review Text": review_text
                    })

                    # Stop collecting if the required number of reviews is reached
                    if len(review_data) >= num_reviews:
                        break

                except Exception as e:
                    print(f"Skipping review due to error: {e}")

            page_number += 1  # Increment page number for tracking

        except Exception as e:
            print(f"Error scraping page {page_number}: {e}")

        time.sleep(2)  # Adding delay to prevent getting blocked

    return pd.DataFrame(review_data)

# Scraping 1000 reviews for Oppenheimer
reviews_df = scrape_oppenheimer_reviews(1000)

# Save the collected reviews to a CSV file
reviews_df.to_csv("oppenheimer_reviews.csv", index=False)

# Display first few rows of the collected data
reviews_df.head()



,Review Title,Review Text
0,Murphy is exceptional,You'll have to have your wits about you and yo...
1,"A challenging watch to be sure, but a worthwhi...",No Text
2,Quality but exhausting,I'm a big fan of Nolan's work so was really lo...
3,And the Oscar goes to...,I'm still collecting my thoughts after experie...
4,A brilliantly layered examination of a man thr...,"""Oppenheimer"" is a biographical thriller film ..."


In [2]:
from google.colab import files

# Save the collected reviews to a CSV file
reviews_df.to_csv("oppenheimer_reviews.csv", index=False)

# Download the CSV file
files.download("oppenheimer_reviews.csv")

# Display first few rows of the collected data
reviews_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Review Title,Review Text
0,Murphy is exceptional,You'll have to have your wits about you and yo...
1,"A challenging watch to be sure, but a worthwhi...",No Text
2,Quality but exhausting,I'm a big fan of Nolan's work so was really lo...
3,And the Oscar goes to...,I'm still collecting my thoughts after experie...
4,A brilliantly layered examination of a man thr...,"""Oppenheimer"" is a biographical thriller film ..."


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [5]:
# Write code for each of the sub parts with proper comments.
import pandas as pd
import re

# Load the dataset
file_path = "oppenheimer_reviews.csv"
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())

# Function to remove special characters and punctuations
df["Cleaned Review"] = df["Review Text"].astype(str).apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Display first few rows after cleaning
print(df.head())
# Function to remove numbers
df["Cleaned Review"] = df["Cleaned Review"].apply(lambda x: re.sub(r'\d+', '', x))

# Display first few rows after cleaning
print(df.head())
# Manually defined stopwords list
manual_stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
    "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"
])

# Function to remove stopwords
def remove_manual_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in manual_stopwords]
    return " ".join(filtered_words)

# Apply stopword removal
df["Cleaned Review"] = df["Cleaned Review"].apply(remove_manual_stopwords)

# Display first few rows after cleaning
print(df.head())
# Convert text to lowercase
df["Cleaned Review"] = df["Cleaned Review"].str.lower()

# Display first few rows after cleaning
print(df.head())
import nltk
from nltk.stem import PorterStemmer

# Initialize Porter Stemmer
porter = PorterStemmer()

# Function to apply stemming
def apply_stemming(text):
    words = text.split()
    stemmed_words = [porter.stem(word) for word in words]
    return " ".join(stemmed_words)

# Apply stemming
df["Stemmed Review"] = df["Cleaned Review"].apply(apply_stemming)

# Display first few rows after cleaning
print(df.head())
import nltk
from nltk.stem import PorterStemmer

# Initialize Porter Stemmer
porter = PorterStemmer()

# Function to apply stemming
def apply_stemming(text):
    words = text.split()
    stemmed_words = [porter.stem(word) for word in words]
    return " ".join(stemmed_words)

# Apply stemming
df["Stemmed Review"] = df["Cleaned Review"].apply(apply_stemming)

# Display first few rows after cleaning
print(df.head())
!pip install spacy
import spacy

# Load the English NLP model
nlp = spacy.blank("en")

# Function to apply lemmatization using spaCy
def apply_spacy_lemmatization(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return " ".join(lemmatized_words)

# Apply lemmatization
df["Lemmatized Review"] = df["Cleaned Review"].apply(apply_spacy_lemmatization)

# Display first few rows after cleaning
print(df.head())
from google.colab import files

# Save the cleaned dataset to a new CSV file
cleaned_file_path = "oppenheimer_reviews_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Download the file
files.download(cleaned_file_path)
from google.colab import files

# Save the cleaned dataset to a new CSV file
cleaned_file_path = "oppenheimer_reviews_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Download the file
files.download(cleaned_file_path)

                                        Review Title  \
0                              Murphy is exceptional   
1  A challenging watch to be sure, but a worthwhi...   
2                             Quality but exhausting   
3                           And the Oscar goes to...   
4  A brilliantly layered examination of a man thr...   

                                         Review Text  
0  You'll have to have your wits about you and yo...  
1                                            No Text  
2  I'm a big fan of Nolan's work so was really lo...  
3  I'm still collecting my thoughts after experie...  
4  "Oppenheimer" is a biographical thriller film ...  
                                        Review Title  \
0                              Murphy is exceptional   
1  A challenging watch to be sure, but a worthwhi...   
2                             Quality but exhausting   
3                           And the Oscar goes to...   
4  A brilliantly layered examination of a man thr... 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [6]:
# Your code here

# Loading Cleaned dataset
import pandas as pd

# Load the cleaned dataset
file_path = "oppenheimer_reviews_cleaned.csv"  # Update with the correct path if needed
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())
# Rule-Based POS Tagging
import re
from collections import Counter

# Define regex patterns for basic POS categories
noun_pattern = r'\b(?:man|woman|movie|film|story|performance|actor|scene|director|cinematography|dialogue|character|plot)\b'
verb_pattern = r'\b(?:is|was|are|were|has|had|do|does|did|makes|made|watch|watched|perform|performed|act|acted|direct|directed)\b'
adj_pattern = r'\b(?:good|bad|amazing|great|fantastic|boring|interesting|intense|emotional|visual|outstanding)\b'
adv_pattern = r'\b(?:quickly|slowly|beautifully|strongly|greatly|intensely|emotionally|visually)\b'

# Function to count POS tags using regex
def rule_based_pos_count(text):
    noun_count = len(re.findall(noun_pattern, text, re.IGNORECASE))
    verb_count = len(re.findall(verb_pattern, text, re.IGNORECASE))
    adj_count = len(re.findall(adj_pattern, text, re.IGNORECASE))
    adv_count = len(re.findall(adv_pattern, text, re.IGNORECASE))

    return {"Noun Count": noun_count, "Verb Count": verb_count, "Adjective Count": adj_count, "Adverb Count": adv_count}

# Apply rule-based POS tagging
pos_results = df["Cleaned Review"].dropna().apply(rule_based_pos_count)

# Aggregate counts
total_counts = Counter()
for counts in pos_results:
    for key, value in counts.items():
        total_counts[key] += value

# Convert to DataFrame
pos_summary_df = pd.DataFrame([total_counts])

# Display POS summary
print(pos_summary_df)
# Rule-Based Constituency Parsing
# Define a simple constituency parsing function
def rule_based_constituency_parse(text):
    words = text.split()
    if len(words) < 6:
        return "(S (NP " + " ".join(words) + "))"
    return f"(S (NP {words[0]} {words[1]}) (VP {words[2]} {words[3]}) (NP {words[4]} {words[5]}))"

# Apply constituency parsing to one sample sentence
sample_text = df["Cleaned Review"].dropna().iloc[0]
constituency_tree = rule_based_constituency_parse(sample_text)

# Display the parsed result
print("Constituency Tree:", constituency_tree)
# Rule-Based Dependency Parsing
# Define a simple dependency parsing function
def rule_based_dependency_parse(text):
    words = text.split()
    dependencies = [(words[i], "modifies", words[i+1]) for i in range(len(words)-1)]
    return dependencies

# Apply dependency parsing to the same sample sentence
dependency_tree = rule_based_dependency_parse(sample_text)

# Display the parsed result
print("Dependency Tree:", dependency_tree)
# Rule-Based Named Entity Recognition (NER)
# Define a simple rule-based Named Entity Recognition (NER) function
def rule_based_ner(text):
    person_pattern = r'\b(?:oppenheimer|nolan|murphy|cillian|emily|rdj|florence|pugh)\b'
    org_pattern = r'\b(?:imdb|warner bros|universal|hollywood|academy awards|oscars)\b'
    loc_pattern = r'\b(?:germany|usa|britain|london|paris|new york|los angeles)\b'
    date_pattern = r'\b(?:\d{4}|january|february|march|april|may|june|july|august|september|october|november|december)\b'

    person_count = len(re.findall(person_pattern, text, re.IGNORECASE))
    org_count = len(re.findall(org_pattern, text, re.IGNORECASE))
    loc_count = len(re.findall(loc_pattern, text, re.IGNORECASE))
    date_count = len(re.findall(date_pattern, text, re.IGNORECASE))

    return {"Person Count": person_count, "Organization Count": org_count, "Location Count": loc_count, "Date Count": date_count}

# Apply rule-based NER to all reviews
ner_results = df["Cleaned Review"].dropna().apply(rule_based_ner)

# Aggregate counts
total_ner_counts = Counter()
for counts in ner_results:
    for key, value in counts.items():
        total_ner_counts[key] += value

# Convert to DataFrame
ner_summary_df = pd.DataFrame([total_ner_counts])

# Display Named Entity Recognition summary
print(ner_summary_df)

                                        Review Title  \
0                              Murphy is exceptional   
1  A challenging watch to be sure, but a worthwhi...   
2                             Quality but exhausting   
3                           And the Oscar goes to...   
4  A brilliantly layered examination of a man thr...   

                                         Review Text  \
0  You'll have to have your wits about you and yo...   
1                                            No Text   
2  I'm a big fan of Nolan's work so was really lo...   
3  I'm still collecting my thoughts after experie...   
4  "Oppenheimer" is a biographical thriller film ...   

                                      Cleaned Review  \
0  youll wits brain fully switched watching oppen...   
1                                               text   
2  im big fan nolans work really looking forward ...   
3  im still collecting thoughts experiencing film...   
4  oppenheimer biographical thriller film writ

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [7]:
import requests
import pandas as pd
import time

# GitHub API URL for searching repositories tagged as "github-action"
GITHUB_API_URL = "https://api.github.com/search/repositories"

# Headers for authentication & bot protection
HEADERS = {
    "Accept": "application/vnd.github.v3+json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def scrape_github_actions_api(max_pages=5):
    """
    Fetches GitHub Actions repositories using the GitHub API.
    Extracts:
    - Repository Name
    - Description
    - URL
    """

    actions_data = []  # List to store results
    per_page = 30  # GitHub API allows 30 results per page

    for page in range(1, max_pages + 1):

        # Query for repositories with the topic "github-action"
        params = {
            "q": "topic:github-action",
            "sort": "stars",
            "order": "desc",
            "per_page": per_page,
            "page": page
        }

        response = requests.get(GITHUB_API_URL, headers=HEADERS, params=params)

        if response.status_code == 200:
            data = response.json()
            if not data.get("items"):
                break  # Stop if no data is found

            for repo in data["items"]:
                actions_data.append({
                    "Product Name": repo["name"],
                    "Description": repo["description"] if repo["description"] else "No Description Available",
                    "URL": repo["html_url"],
                    "Page Number": page
                })

        else:
            print(f"Failed to retrieve page {page}. Status Code: {response.status_code}")
            break  # Stop on failure

        time.sleep(2)  # Add delay to avoid rate limits

    return pd.DataFrame(actions_data)

# Scrape and save GitHub Actions using API
actions_df = scrape_github_actions_api(10)
actions_df.to_csv("github_actions_api.csv", index=False)

# Display first few rows of the collected data
actions_df.head(5)

,Product Name,Description,URL,Page Number
0,metrics,📊 An infographics generator with 30+ plugins a...,https://github.com/lowlighter/metrics,1
1,github-pages-deploy-action,🚀 Automatically deploy your project to GitHub ...,https://github.com/JamesIves/github-pages-depl...,1
2,action-tmate,Debug your GitHub Actions via SSH by using tma...,https://github.com/mxschmitt/action-tmate,1
3,github-profile-summary-cards,A tool to generate your github summary card fo...,https://github.com/vn7n24fzkq/github-profile-s...,1
4,create-pull-request,A GitHub action to create a pull request for c...,https://github.com/peter-evans/create-pull-req...,1


In [8]:
from google.colab import files

# Save the collected reviews to a CSV file
actions_df.to_csv("github_actions_api.csv", index=False)

# Download the CSV file
files.download("github_actions_api.csv")

# Display first few rows of the collected data
actions_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Product Name,Description,URL,Page Number
0,metrics,📊 An infographics generator with 30+ plugins a...,https://github.com/lowlighter/metrics,1
1,github-pages-deploy-action,🚀 Automatically deploy your project to GitHub ...,https://github.com/JamesIves/github-pages-depl...,1
2,action-tmate,Debug your GitHub Actions via SSH by using tma...,https://github.com/mxschmitt/action-tmate,1
3,github-profile-summary-cards,A tool to generate your github summary card fo...,https://github.com/vn7n24fzkq/github-profile-s...,1
4,create-pull-request,A GitHub action to create a pull request for c...,https://github.com/peter-evans/create-pull-req...,1


In [9]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv("github_actions_api.csv")

# Drop duplicate rows
df = df.drop_duplicates()

# Drop rows where 'Product Name' or 'Description' is missing
df = df.dropna(subset=['Product Name', 'Description'])

# Define a function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
        return text
    return ""

# Apply text cleaning to relevant columns
df['Product Name'] = df['Product Name'].apply(clean_text)
df['Description'] = df['Description'].apply(clean_text)

# Save cleaned data to a new CSV file
df.to_csv("github_actions_api_cleaned.csv", index=False)

# Display final dataset info
print("Data cleaning completed. Cleaned dataset saved as 'github_actions_cleaned.csv'")
df.head()

Data cleaning completed. Cleaned dataset saved as 'github_actions_cleaned.csv'


,Product Name,Description,URL,Page Number
0,metrics,an infographics generator with 30 plugins and ...,https://github.com/lowlighter/metrics,1
1,githubpagesdeployaction,automatically deploy your project to github pa...,https://github.com/JamesIves/github-pages-depl...,1
2,actiontmate,debug your github actions via ssh by using tma...,https://github.com/mxschmitt/action-tmate,1
3,githubprofilesummarycards,a tool to generate your github summary card fo...,https://github.com/vn7n24fzkq/github-profile-s...,1
4,createpullrequest,a github action to create a pull request for c...,https://github.com/peter-evans/create-pull-req...,1


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [10]:
!pip install tweepy pandas

In [14]:
!pip install dotenv

In [15]:
import tweepy
import pandas as pd
import re
import os
from dotenv import load_dotenv

# Load API keys securely from .env file
load_dotenv()

API_BEARER_TOKEN="AAAAAAAAAAAAAAAAAAAAAIZ%2BzQEAAAAAzdgNKq%2BK4aZc86U3IsFVFu8qoIU%3DCIjrUXTR4U0jXmp2oj27uiQuOf0Qb5VAdUVcyZeh6JU8DOkvhz"

# Authenticate using Twitter API v2 (Ensure the Bearer Token is correct)
client = tweepy.Client(bearer_token=API_BEARER_TOKEN)

print("✅ Twitter API Authentication Successful")

✅ Twitter API Authentication Successful


In [16]:
# Define search query (Twitter API v2 uses different parameters)
SEARCH_QUERY = "(Machine Learning OR Artificial Intelligence) -is:retweet lang:en"
TWEET_COUNT = 100  # Number of tweets to retrieve

# Function to scrape tweets using Twitter API v2
def scrape_tweets_v2(query, count):
    tweets_data = []

    response = client.search_recent_tweets(query=query, max_results=100, tweet_fields=["id", "text", "author_id"])

    if response.data:
        for tweet in response.data:
            tweets_data.append({
                "Tweet ID": tweet.id,
                "Username": tweet.author_id,  # API v2 does not provide usernames directly
                "Text": tweet.text
            })

    return pd.DataFrame(tweets_data)

# Scrape tweets
tweets_df = scrape_tweets_v2(SEARCH_QUERY, TWEET_COUNT)

# Save raw tweets
tweets_df.to_csv("raw_tweets.csv", index=False)
print("✅ Raw tweets saved as 'raw_tweets.csv'")

✅ Raw tweets saved as 'raw_tweets.csv'


In [17]:
# Download the CSV file
files.download("raw_tweets.csv")

# Display first few rows of the collected data
tweets_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Tweet ID,Username,Text
0,1892381149856444423,1891831704463290368,🔍 What makes ChessMaster AI a game-changer? Ou...
1,1892381127785971982,1264214958843334658,Biggest-ever AI biology model writes DNA on de...
2,1892381021389132090,35829818,Everyone should be getting up to speech on art...
3,1892381001977852204,1522203426947538945,Star power on Earth: US scientists crack nucle...
4,1892380966368260378,1708688942575321089,[What is AI (DeepSeek vs. OpenAI) #654131]{(8....


In [18]:
# Function to clean tweet text
def clean_tweet(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"#\w+", "", text)  # Remove hashtags
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Apply cleaning
tweets_df["Cleaned Text"] = tweets_df["Text"].apply(clean_tweet)

# Save cleaned tweets
tweets_df.to_csv("cleaned_tweets.csv", index=False)
print("✅ Cleaned tweets saved as 'cleaned_tweets.csv'")
# Display first few rows of the collected data
tweets_df.head()

✅ Cleaned tweets saved as 'cleaned_tweets.csv'


,Tweet ID,Username,Text,Cleaned Text
0,1892381149856444423,1891831704463290368,🔍 What makes ChessMaster AI a game-changer? Ou...,what makes chessmaster ai a gamechanger our ai...
1,1892381127785971982,1264214958843334658,Biggest-ever AI biology model writes DNA on de...,biggestever ai biology model writes dna on dem...
2,1892381021389132090,35829818,Everyone should be getting up to speech on art...,everyone should be getting up to speech on art...
3,1892381001977852204,1522203426947538945,Star power on Earth: US scientists crack nucle...,star power on earth us scientists crack nuclea...
4,1892380966368260378,1708688942575321089,[What is AI (DeepSeek vs. OpenAI) #654131]{(8....,what is ai deepseek vs openai 81 what is ai th...


In [19]:
# Remove duplicates
tweets_df.drop_duplicates(inplace=True)

# Remove missing values
tweets_df.dropna(inplace=True)

# Save final cleaned data
tweets_df.to_csv("final_cleaned_tweets.csv", index=False)
print("✅ Final cleaned tweets saved as 'final_cleaned_tweets.csv'")

# Display first few rows
tweets_df.head()

✅ Final cleaned tweets saved as 'final_cleaned_tweets.csv'


,Tweet ID,Username,Text,Cleaned Text
0,1892381149856444423,1891831704463290368,🔍 What makes ChessMaster AI a game-changer? Ou...,what makes chessmaster ai a gamechanger our ai...
1,1892381127785971982,1264214958843334658,Biggest-ever AI biology model writes DNA on de...,biggestever ai biology model writes dna on dem...
2,1892381021389132090,35829818,Everyone should be getting up to speech on art...,everyone should be getting up to speech on art...
3,1892381001977852204,1522203426947538945,Star power on Earth: US scientists crack nucle...,star power on earth us scientists crack nuclea...
4,1892380966368260378,1708688942575321089,[What is AI (DeepSeek vs. OpenAI) #654131]{(8....,what is ai deepseek vs openai 81 what is ai th...


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

While doing first question scraping one. I found difficulty in that. But after understanding that felt better. For twiiter question also I faced some issue in solving it.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog